In [1]:
from collections import namedtuple
from sage.rings.polynomial.polydict import ETuple
load('utils.sage') # log2_strict, MerkleTree, Transcript

In [2]:
F = GF(15*2^27+1); print(F)
R.<X> = F[]

Finite Field of size 2013265921


In [3]:
def to_bits(x, n_bits):
    for i in range(n_bits):
        yield (x >> i) & 1

# The boolean hypercube
def B(n_bits):
    for i in range(2^n_bits):
        yield list(to_bits(i, n_bits))

In [4]:
def eq(ys, xs=None, R=None):
    ys = list(ys)
    if xs is None:
        if R is None:
            R = PolynomialRing(ys[0].parent(), len(ys), 'x')
        xs = list(R.gens())
    assert len(xs) == len(ys)
    return product(
        x*y + (1 - x)*(1 - y)
        for x, y in zip(xs, ys)
    )

def mle(evals):
    n_bits = log2_strict(len(evals))
    return sum(eq(to_bits(i, n_bits))*evals[i] for i in range(2^n_bits))

x = matrix.random(F, 1, 16).row(0)
assert x == vector([mle(x)(b) for b in B(4)])

In [5]:
def rs_matrix(field, msg_bits, rate_bits):
    if msg_bits == 0:
        return matrix.ones(field, 1, 2^rate_bits)
    inner = rs_matrix(field, msg_bits-1, rate_bits)
    sg = two_adic_subgroup(field, msg_bits+rate_bits)
    T1, T2 = [matrix.diagonal(d) for d in (sg[:len(sg)/2], sg[len(sg)/2:])]
    return matrix.block([[inner, inner], [inner * T1, inner * T2]], subdivide=False)

print(rs_matrix(F, 2, 1))
print('vs:')
print(codes.ReedSolomonCode(F, 8, 4).generator_matrix())

[         1          1          1          1          1          1          1          1]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]
vs:
[         1          1          1          1          1          1          1          1]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]


In [6]:
def multilinear_coeffs(p):
    d = p.dict()
    return vector([
        d.get(ETuple(b), 0)
        for b in B(len(p.parent().gens()))
    ])

In [7]:
def encode(v, rate_bits):
    n_bits = log2_strict(len(v))
    field = v.base_ring()
    return v * rs_matrix(field, n_bits, rate_bits)

def interpolate2(xa, ya, xb, yb, x):
    return ya + (yb - ya) * (x - xa) / (xb - xa)

def fold_evals(v, beta):
    # diag(T1) = (g^0, .., g^n/2), diag(T2) = (g^(n/2+1), .., g^(n-1))
    n_bits = log2_strict(len(v))
    sg = two_adic_subgroup(F, n_bits)
    T1, T2 = sg[:len(v)/2], sg[len(v)/2:]
    return vector([
        interpolate2(T1[j], v[j], T2[j], v[j+len(v)/2], beta)
        for j in range(len(v)/2)
    ])

In [16]:
EvalProof = namedtuple('EvalProof', [
    'comms', 'y', 'hs'
])

def basefold_prove(t, evals, rate_bits):
    n_vars = log2_strict(len(evals))
    f = mle(evals)
    codeword = encode(multilinear_coeffs(f), rate_bits)
    oracles = [MerkleTree(codeword)]
    t.observe(oracles[0].root())
    z = [t.challenge(F) for _ in range(n_vars)]
    y = f(z)
    print(z, y)
    hs = [sum(f(b+[X]) * eq(z)(b+[X]) for b in B(n_vars-1))]
    rs = []
    for i in range(n_vars):
        rs.insert(0, t.challenge(F))
        codeword = fold_evals(codeword, rs[0])
        oracles.append(MerkleTree(codeword))
        t.observe(oracles[-1].root())
        if i < n_vars - 1:
            hs.append(sum(
                f(b+[X]+rs) * eq(z)(b+[X]+rs)
                for b in B(n_vars-i-2)
            ))
    print(f'{codeword=} {rs=} {f(rs)=}')
    return EvalProof(comms=[o.root() for o in oracles], y=y, hs=hs)

def basefold_verify(t, proof, n_vars, rate_bits):
    comms, y, hs = proof
    t.observe(comms[0])
    z = [t.challenge(F) for _ in range(n_vars)]
    print(z, y)
    rs = []
    old_eval = y
    for comm, h in zip(comms[1:], hs):
        assert h(0) + h(1) == old_eval
        rs.insert(0, t.challenge(F))
        old_eval = h(rs[0])
        t.observe(comm)
    final_eval = old_eval / eq(rs, z)
    print(f'{final_eval=}')
    assert MerkleTree(encode(vector([final_eval]), 1)).root() == comms[-1]

# Parameters
n_vars = 5
rate_bits = 1
with seed(0):
    evals = matrix.random(F, 1, 2^n_vars).row(0)
print(len(evals))

proof = basefold_prove(Transcript(), evals, rate_bits)
print(); print(proof); print()
basefold_verify(Transcript(), proof, n_vars, rate_bits)

32
[66809542, 437403465, 1852014596, 1931687671, 1847338564] 1413980145
codeword=(1316390028, 1316390028) rs=[1349761488, 225760585, 38033045, 17951029, 455585381] f(rs)=1316390028

EvalProof(comms=[b'fe94afbb', b'43b6cbe2', b'4779567f', b'bd9de4fc', b'46584e2a', b'd4ab64f5'], y=1413980145, hs=[611272261*X^2 + 671153020*X + 65777432, 1270798613*X^2 + 479288763*X + 1005010434, 1820313339*X^2 + 929239804*X + 1369825294, 1137047125*X^2 + 348446020*X + 1599670240, 271217647*X^2 + 1578584800*X + 1381443804])

[66809542, 437403465, 1852014596, 1931687671, 1847338564] 1413980145
final_eval=1316390028


In [30]:
t = Transcript()
trace_oracle = MerkleTree(codeword)
t.observe(trace_oracle.root())
z = [t.challenge(F) for _ in range(n_vars)]
y = f(z)
z, y

NameError: name 'codeword' is not defined

In [122]:
h_d = sum(f(b+[X]) * eq(b+[X], z) for b in B(n_vars-1)); h_d

455260732*X^2 + 1561868076*X + 1659258104

In [74]:
list(B(3))

[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 1, 0],
 [0, 0, 1],
 [1, 0, 1],
 [0, 1, 1],
 [1, 1, 1]]

In [89]:
f.variables()

(x0, x1, x2, x3)

In [127]:
f(list(B(n_vars-1))[5] + [X])

1279983006*X + 40893827

In [128]:
eq(list(B(n_vars-1))[2] + [X], z)

1811553886*X + 17472635

In [130]:
list(range(5, -1, -1))

[5, 4, 3, 2, 1, 0]

In [132]:
list(reversed(range(5)))

[4, 3, 2, 1, 0]

In [135]:
m = codes.ReedSolomonCode(F, 8, 4).generator_matrix(); m

[         1          1          1          1          1          1          1          1]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]

In [142]:
import inspect
print(inspect.getsource(codes.ReedSolomonCode(F, 8, 4).encode))

    def encode(self, word, encoder_name=None, *args, **kwargs):
        r"""
        Transforms an element of a message space into a codeword.

        INPUT:

        - ``word`` -- an element of a message space of the code

        - ``encoder_name`` -- (default: ``None``) Name of the encoder which will be used
          to encode ``word``. The default encoder of ``self`` will be used if
          default value is kept.

        - ``args``, ``kwargs`` -- all additional arguments are forwarded to the construction of the
          encoder that is used..

        One can use the following shortcut to encode a word ::

            C(word)

        OUTPUT:

        - a vector of ``self``.

        EXAMPLES::

            sage: G = Matrix(GF(2), [[1,1,1,0,0,0,0], [1,0,0,1,1,0,0],
            ....:                    [0,1,0,1,0,1,0], [1,1,0,1,0,0,1]])
            sage: C = LinearCode(G)
            sage: word = vector((0, 1, 1, 0))
            sage: C.encode(word)
            (1, 1, 0, 0, 1,

In [166]:
e = codes.ReedSolomonCode(F, 4, 4).encoder()
e.unencode_nocheck

<bound method Encoder.unencode_nocheck of Evaluation vector-style encoder for [4, 4, 1] Reed-Solomon Code over GF(2013265921)>

In [83]:
c = codes.ReedSolomonCode(F, 4, 4); print(c.generator_matrix())
v = matrix.random(F, 1, 4).row(0); print(v)
codeword = c.encode(v); codeword, v * c.generator_matrix()

[         1          1          1          1]
[         1 1728404513 2013265920  284861408]
[         1 2013265920          1 2013265920]
[         1  284861408 2013265920 1728404513]
(1872393269, 737778300, 1877566767, 483611214)


((944817708, 1802853353, 515304601, 200065572),
 (944817708, 1802853353, 515304601, 200065572))

In [168]:
c.generator_matrix() * codeword

(1516158824, 1315000374, 1649604317, 590442052)

In [169]:
e._unencoder_matrix()

(
[1509949441 1509949441 1509949441 1509949441]              
[1509949441   71215352  503316480 1942050569]              
[1509949441  503316480 1509949441  503316480]              
[1509949441 1942050569  503316480   71215352], (0, 1, 2, 3)
)

In [171]:
c.generator_matrix().inverse()

[1509949441 1509949441 1509949441 1509949441]
[1509949441   71215352  503316480 1942050569]
[1509949441  503316480 1509949441  503316480]
[1509949441 1942050569  503316480   71215352]

In [159]:
c.generator_matrix().matrix_from_columns(range(4)).inverse() * codeword[:4]

(1928653012, 1165738772, 678206814, 763383896)

In [172]:
c.generator_matrix().inverse() * c.generator_matrix()
c.generator_matrix().inverse() * c.generator_matrix()

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]

In [177]:
matrix.block_diagonal([matrix.column(F, [1, 5])] * 3)

[1|0|0]
[5|0|0]
[-+-+-]
[0|1|0]
[0|5|0]
[-+-+-]
[0|0|1]
[0|0|5]

In [190]:
idft_matrix(F, 2) * dft_matrix(F, 2, 1)

[         1 1889756441          0   18921224          0  987711737          0 1130142441]
[         0 1130142441          1 1889756441          0   18921224          0  987711737]
[         0  987711737          0 1130142441          1 1889756441          0   18921224]
[         0   18921224          0  987711737          0 1130142441          1 1889756441]

In [192]:
lde = v * idft_matrix(F, 2) * dft_matrix(F, 2, 1); lde

(379039706, 301809050, 147610513, 1927575377, 1922350520, 1480314799, 1335383054, 74684567)

In [198]:
lde2 = lde * idft_matrix(F, 3) * folding_matrix(F, 2, 5) * dft_matrix(F, 2); lde2

(1318949920, 1815838602, 332778774, 1849156013)

In [199]:
lde3 = lde2 * idft_matrix(F, 2) * folding_matrix(F, 1, 5) * dft_matrix(F, 1); lde3

(1278026291, 1278026291)

In [200]:
idft_matrix(F, 3) * folding_matrix(F, 2, 5) * dft_matrix(F, 2)

[         3          0          0          0]
[         0  477324976          0          0]
[         0          0 1718786481          0]
[         0          0          0 1052249268]
[2013265919          0          0          0]
[         0 1535940946          0          0]
[         0          0  294479441          0]
[         0          0          0  961016654]

In [80]:
c = codes.ReedSolomonCode(F, 4, 4); c.generator_matrix()

[         1          1          1          1]
[         1 1728404513 2013265920  284861408]
[         1 2013265920          1 2013265920]
[         1  284861408 2013265920 1728404513]

In [86]:
c = codes.ReedSolomonCode(F, 4, 2); c.generator_matrix()

[         1          1          1          1]
[         1 1728404513 2013265920  284861408]

In [84]:
c = codes.ReedSolomonCode(F, 8, 4); c.generator_matrix()

[         1          1          1          1          1          1          1          1]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]

In [87]:
G0 = codes.ReedSolomonCode(F, 2, 1).generator_matrix(); G0

[1 1]

In [97]:
sg = two_adic_subgroup(F, 2); print(sg)
T1, T2 = matrix.diagonal(sg[:len(sg)/2]), matrix.diagonal(sg[len(sg)/2:])
G1 = matrix.block([
    [G0, G0],
    [G0 * T1, G0 * T2],
])
G1

[1, 1728404513, 2013265920, 284861408]


[         1          1|         1          1]
[---------------------+---------------------]
[         1 1728404513|2013265920  284861408]

In [99]:
sg = two_adic_subgroup(F, 3); print(sg)
T1, T2 = matrix.diagonal(sg[:len(sg)/2]), matrix.diagonal(sg[len(sg)/2:])
G2 = matrix.block([
    [G1, G1],
    [G1 * T1, G1 * T2],
], subdivide=False)
G2

[1, 1592366214, 1728404513, 211723194, 2013265920, 420899707, 284861408, 1801542727]


[         1          1          1          1          1          1          1          1]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]

In [108]:
def rs_matrix(field, msg_bits, rate_bits):
    if msg_bits == 0:
        return matrix.ones(field, 1, 2^rate_bits)
    inner = rs_matrix(field, msg_bits-1, rate_bits)
    sg = two_adic_subgroup(field, msg_bits+rate_bits)
    T1, T2 = [matrix.diagonal(d) for d in (sg[:len(sg)/2], sg[len(sg)/2:])]
    return matrix.block([[inner, inner], [inner * T1, inner * T2]], subdivide=False)

print(rs_matrix(F, 0, 1))
print(rs_matrix(F, 1, 1))
print(rs_matrix(F, 2, 1))

[1 1]
[         1          1          1          1]
[         1 1728404513 2013265920  284861408]
[         1          1          1          1          1          1          1          1]
[         1 1728404513 2013265920  284861408          1 1728404513 2013265920  284861408]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
[         1  211723194  284861408 1592366214 2013265920 1801542727 1728404513  420899707]


In [109]:
print(rs_matrix(F, 0, 2))
print(rs_matrix(F, 1, 2))

[1 1 1 1]
[         1          1          1          1          1          1          1          1]
[         1 1592366214 1728404513  211723194 2013265920  420899707  284861408 1801542727]
